In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

0 - No such habit
1- habit currently practiced
2 - past habit now has stopped (minimum 6 months)

In [ ]:
str_habit_false = "0 - No such habit"
lfall.select(
  pl.col(*list_id_cols, "HABITS", "TOBACCO", "BBETEL QUID CHEWING", "ALCOHOL")
).with_columns(
  pl.when(pl.col("TOBACCO").is_null() | pl.col("TOBACCO").eq(str_habit_false))
  .then(False)
  .otherwise(True)
  .alias("bool_tobacco"),
  pl.when(
    pl.col("BBETEL QUID CHEWING").is_null()
    | pl.col("BBETEL QUID CHEWING").eq(str_habit_false)
  )
  .then(False)
  .otherwise(True)
  .alias("bool_betel"),
  pl.when(pl.col("ALCOHOL").is_null() | pl.col("ALCOHOL").eq(str_habit_false))
  .then(False)
  .otherwise(True)
  .alias("bool_alcohol"),
).with_columns(
  pl.any_horizontal(["bool_tobacco", "bool_betel", "bool_alcohol"]).alias("bool_any")
).filter(pl.col("HABITS") != pl.col("bool_any")).collect()
